# Creación de conversaciones ARP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación ARP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [ ]:
import random
import re
import pickle
import math
import time
import os 

import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether, ARP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de una comunicación ARP

<center><img src="https://lh3.googleusercontent.com/proxy/5BFPjEuLzirf_rQUpK9oPibFDhfXjP7eez9zJdPXfEl5QZyEp6jLTDSpVc8RBygmwlQ5s4hV-V_muXLdOGmmrsjD-JaqO4HtlQHkO1D8Tj8">
    
<center> Esta imagen es genérica, nosotros crearemos conversaciones que sean tanto una pregunta a broadcast como a una ubicación específica.
<br>
    ---
<br>Una conversación normal ARP tiene una petición (opcode = 1) en la que una fuente pregunta a broadcast o a otro elemento de la red por una IP en concreto, y diciendo la IP a la que comunicar la respuesta.
La respuesta tiene (opcode = 2), y tiene como fuente las respuestas, mientras que como destino tiene al que inicia la conversación.   
<center>---

La primera query (query1) representa una conexión ARP sencilla donde se envía la petición a un equipo específico, la segunda (query2) es más complicada, enviando la solicitud a broadcast.

In [2]:
system_message1 = "This is a ARP conversation summary:\n"
system_message1 += 'Source: MAC="c2:3d:19:6c:00:01", IP="10.0.0.1" // Destination: MAC="c2:3c:19:6c:00:01" // Wanted: IP= "10.0.0.2"\n'
system_message1 += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
system_message1 += 'pkt_1=scapy.Ether(src="c2:3d:19:6c:00:01", dst="c2:3c:19:6c:00:01")/scapy.ARP(op=1, pdst="10.0.0.2", psrc="10.0.0.1", hwdst="c2:3c:19:6c:00:01", hwsrc="c2:3d:19:6c:00:01")\n'
system_message1 += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message1 += 'pkt_2=scapy.Ether(src="c2:3c:19:6c:00:01", dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="c2:3d:19:6c:00:01",  pdst="10.0.0.1")\n'

In [3]:
system_message2 = "This is a ARP conversation summary:\n"
system_message2 += 'Source: MAC="c2:3d:19:6c:00:01", IP="10.0.0.1" // Destination: MAC="FF:FF:FF:FF:FF:FF" // Wanted: IP= "10.0.0.2"\n'
system_message2 += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
system_message2 += 'pkt_1=scapy.Ether(src="c2:3d:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="10.0.0.2", psrc="10.0.0.1", hwdst="00:00:00:00:00:00", hwsrc="c2:3d:19:6c:00:01")\n'
system_message2 += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message2 += 'rand_MAC= "02:00:00:%02x:%02x:%02x" % (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))'
system_message2 += 'pkt_2=scapy.Ether(src=rand_MAC, dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc=rand_MAC, hwdst="c2:3d:19:6c:00:01",  pdst="10.0.0.1")\n'
system_message2 += 'Take your time (few seconds) to think if the packets and their variables make sense for an ARP converstaion, if not, correct them\n'

## Carga de posibles conversaciones

In [8]:
with open("../../data/new/ARP_Conv_Testing_v2.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()

print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 305


### Carga de posibles respuestas anteriores para encadenarlas

In [9]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [10]:
try:
    responses = pickle.load( open("../../data/Conversations/pickle/ARP_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


## Generación de comandos de paquetes de conversación

In [11]:
for i in range(200):
   
    summary = Conv_summaries[i]
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    #summaries = random.choices(Conv_summary,k=summaries_per_request)
    text_sum = ""
    
    if summary.find('FF:FF:FF:FF:FF:FF') == -1: # No esta, es decir, entre disps

        query_content = "This is a conversation summary:\n"
        query_content += summary + "\n"
        text_sum += summary + "\n"
        query_content += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.\n\n"
        query_content += "Use the following structure:\n"
        query_content += "pkt_1=...\n"
        query_content += "time.sleep(abs(random.gauss(0, 0.03)))\n"
        query_content += "pkt_2=...\n"
        
        #Query es el mensaje que le envias al modelo
        print(query_content)

        messages=[{"role": "system", "content": system_message1},
        {"role": "user", "content": query_content}]
    
        #Esta es la petición que le mandas al modelo de IA
        completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    else: #si esta, es decir, broadcast

        query_content += "This is a conversation summary:\n"
        query_content += summary + "\n"
        text_sum += summary + "\n"
        query_content += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.\n\n"
        query_content += "Use the following structure:\n"
        query_content += "pkt_1=...\n"
        query_content += "time.sleep(random.randint(1,10)/1000000.0)\n"
        query_content += "rand_MAC=...\n"
        query_content += "pkt_2=...\n"
        #Query es el mensaje que le envias al modelo
        print(query_content)

        messages=[{"role": "system", "content": system_message2},
        {"role": "user", "content": query_content}]
    
        #Esta es la petición que le mandas al modelo de IA
        completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/ARP/pickle/ARP_pairs_Aday_v3.pkl", "wb" ) )
    
    codigo = responses[-1].completion.choices[0].message.content
    
    if codigo.startswith("```python"):
        codigo = codigo[len("```python"):].lstrip()  # Elimina ```python al inicio
    if codigo.endswith("```"):
        codigo = codigo[:-len("```")].rstrip()  # Elimina ``` al final
    
    exec(codigo)
    
    # Guardamos los paquetes generados en un pcap
    with open("../../data/Conversations/ARP/pcap/ARP_generated_gpt_4omini_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_1, append=True)
    
    with open("../../data/Conversations/ARP/pcap/ARP_generated_gpt_4omini_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_2, append=True)
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 1)
This is a conversation summary:
Source: MAC= "56:84:7A:BC:32:F1", IP= "192.168.10.1" // Destination: MAC= "4E:16:BD:8F:47:3C" // Wanted: IP= "192.168.10.2"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 1
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 2)
This is a conversation summary:
Source: MAC= "56:84:7A:BC:32:F1", IP= "192.168.10.1" // Destination: MAC= "4E:16:BD:8F:47:3C" // Wanted: IP= "192.168.10.2"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give

...Generated!
Number of completitions done: 10
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 11)
This is a conversation summary:
Source: MAC= "EE:B3:14:6A:F4:7D", IP= "192.168.10.21" // Destination: MAC= "FD:C2:27:8F:02:8E" // Wanted: IP= "192.168.10.22"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 11
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 12)
This is a conversation summary:
Source: MAC= "EE:B3:14:6A:F4:7D", IP= "192.168.10.21" // Destination: MAC= "FD:C2:27:8F:02:8E" // Wanted: IP= "192.168.10.22"

Generate python code for creating the packe

...Generated!
Number of completitions done: 20
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 21)
This is a conversation summary:
Source: MAC= "03:07:0B:0D:11:13", IP= "192.168.3.4" // Destination: MAC= "05:09:0D:11:15:19" // Wanted: IP= "192.168.4.11"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...
This is a conversation summary:
Source: MAC= "06:0E:16:1A:22:26", IP= "192.168.5.7" // Destination: MAC= "FF:FF:FF:FF:FF:FF" // Wanted: IP= "192.168.6.14"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont star

...Generated!
Number of completitions done: 29
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 30)
This is a conversation summary:
Source: MAC= "21:47:7B:85:A5:CD", IP= "192.168.23.34" // Destination: MAC= "23:49:87:95:BD:D9" // Wanted: IP= "192.168.24.51"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 30
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 31)
This is a conversation summary:
Source: MAC= "21:47:7B:85:A5:CD", IP= "192.168.23.34" // Destination: MAC= "23:49:87:95:BD:D9" // Wanted: IP= "192.168.24.51"

Generate python code for creating the packe

...Generated!
Number of completitions done: 39
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 40)
This is a conversation summary:
Source: MAC= "3F:83:DF:FD:F2:2D", IP= "192.168.43.64" // Destination: MAC= "41:85:D9:01:FA:31" // Wanted: IP= "192.168.44.117"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 40
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 41)
This is a conversation summary:
Source: MAC= "3F:83:DF:FD:F2:2D", IP= "192.168.43.64" // Destination: MAC= "41:85:D9:01:FA:31" // Wanted: IP= "192.168.44.117"

Generate python code for creating the pac

...

...Generated!
Number of completitions done: 159
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 160)
This is a conversation summary:
Source: MAC= "A7:53:8F:9D:D7:ED", IP= "192.168.29.170" // Destination: MAC= "A9:59:89:95:DF:F5" // Wanted: IP= "192.168.30.189"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 160
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 161)
This is a conversation summary:
Source: MAC= "A7:53:8F:9D:D7:ED", IP= "192.168.29.170" // Destination: MAC= "A9:59:89:95:DF:F5" // Wanted: IP= "192.168.30.189"

Generate python code for creating t

...Generated!
Number of completitions done: 169
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 170)
This is a conversation summary:
Source: MAC= "09:0F:15:1D:1F:33", IP= "192.168.4.10" // Destination: MAC= "1D:23:25:27:2B:2F" // Wanted: IP= "192.168.5.13"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 170
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 171)
This is a conversation summary:
Source: MAC= "09:0F:15:1D:1F:33", IP= "192.168.4.10" // Destination: MAC= "1D:23:25:27:2B:2F" // Wanted: IP= "192.168.5.13"

Generate python code for creating the packe

...Generated!
Number of completitions done: 179
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 180)
This is a conversation summary:
Source: MAC= "27:39:3D:71:6D:8A", IP= "192.168.14.40" // Destination: MAC= "59:5D:65:69:6D:89" // Wanted: IP= "192.168.15.43"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 180
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 181)
This is a conversation summary:
Source: MAC= "27:39:3D:71:6D:8A", IP= "192.168.14.40" // Destination: MAC= "59:5D:65:69:6D:89" // Wanted: IP= "192.168.15.43"

Generate python code for creating the p

...Generated!
Number of completitions done: 189
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 190)
This is a conversation summary:
Source: MAC= "45:61:65:D5:BD:DA", IP= "192.168.24.70" // Destination: MAC= "95:91:95:9D:AD:C5" // Wanted: IP= "192.168.25.73"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 190
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 191)
This is a conversation summary:
Source: MAC= "45:61:65:D5:BD:DA", IP= "192.168.24.70" // Destination: MAC= "95:91:95:9D:AD:C5" // Wanted: IP= "192.168.25.73"

Generate python code for creating the p

...Generated!
Number of completitions done: 199
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 200)
This is a conversation summary:
Source: MAC= "63:89:8D:39:0D:2A", IP= "192.168.34.100" // Destination: MAC= "D1:B1:D1:D1:E1:F9" // Wanted: IP= "192.168.35.103"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important. Dont start the resposnses wiht ```python This is a flagrant error.

Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 200


In [34]:
# Generate random MAC address
import random
mac = [random.randint(0x00, 0xff) for i in range(6)]
mac[0] = (mac[0] & 0xfc) | 0x02 # Set the second bit to 1 to indicate a locally administered MAC address
mac = ":".join(map(lambda x: "%02x" % x, mac))

# Create request packet
request_pkt = scapy.Ether(src=mac, dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="192.168.4.20", psrc="192.168.4.10", hwdst="00:00:00:00:00:00", hwsrc=mac)

# Create reply packet
reply_pkt = scapy.Ether(src=mac, dst=mac)/scapy.ARP(op=2, psrc="192.168.4.20", hwsrc=mac, hwdst=mac, pdst="192.168.4.10")

# Add packets to list
pkt_list = [request_pkt, reply_pkt]

In [60]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: MAC=“f3:a4:b5:c6:d7:e8”, IP=“172.16.3.10” // Destination: MAC=“FF:FF:FF:FF:FF:FF” // Wanted: IP= “172.16.3.20”



import scapy.all as scapy

# Generate random MAC address for the responding end
import random
mac = [random.randint(0x00, 0xff) for i in range(6)]
mac[0] |= 0x02
mac = ":".join(["%02x" % x for x in mac])

# Create request packet
request_pkt = scapy.Ether(src="f3:a4:b5:c6:d7:e8", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="172.16.3.20", psrc="172.16.3.10", hwdst="00:00:00:00:00:00", hwsrc="f3:a4:b5:c6:d7:e8")

# Create reply packet
reply_pkt = scapy.Ether(src=mac, dst="f3:a4:b5:c6:d7:e8")/scapy.ARP(op=2, psrc="172.16.3.20", hwsrc=mac, hwdst="f3:a4:b5:c6:d7:e8", pdst="172.16.3.10")

# Add packets to list
pkt_list = [request_pkt, reply_pkt]


In [62]:
#Esta es la forma que tiene una respuesta del modelo de IA
print(responses[-1].completion)

Completion(id='cmpl-92FB5D7MemVeJpZlaomyjw2L6FG9x', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\nimport scapy.all as scapy\n\n# Generate random MAC address for the responding end\nimport random\nmac = [random.randint(0x00, 0xff) for i in range(6)]\nmac[0] |= 0x02\nmac = ":".join(["%02x" % x for x in mac])\n\n# Create request packet\nrequest_pkt = scapy.Ether(src="f3:a4:b5:c6:d7:e8", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="172.16.3.20", psrc="172.16.3.10", hwdst="00:00:00:00:00:00", hwsrc="f3:a4:b5:c6:d7:e8")\n\n# Create reply packet\nreply_pkt = scapy.Ether(src=mac, dst="f3:a4:b5:c6:d7:e8")/scapy.ARP(op=2, psrc="172.16.3.20", hwsrc=mac, hwdst="f3:a4:b5:c6:d7:e8", pdst="172.16.3.10")\n\n# Add packets to list\npkt_list = [request_pkt, reply_pkt]')], created=1710322027, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=274, prompt_tokens=455, total_tokens=729))


### Generación de paquetes

In [65]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<Ether  dst=FF:FF:FF:FF:FF:FF src=f3:a4:b5:c6:d7:e8 type=ARP |<ARP  op=who-has hwsrc=f3:a4:b5:c6:d7:e8 psrc=172.16.3.10 hwdst=00:00:00:00:00:00 pdst=172.16.3.20 |>>, <Ether  dst=f3:a4:b5:c6:d7:e8 src=e7:44:42:a0:65:a0 type=ARP |<ARP  op=is-at hwsrc=e7:44:42:a0:65:a0 psrc=172.16.3.20 hwdst=f3:a4:b5:c6:d7:e8 pdst=172.16.3.10 |>>]


In [67]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [36]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/ARP/pcap/ARPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)